# Comparison of DFT solvers

We compare four different approaches for solving the DFT minimisation problem,
namely a density-based SCF, a potential-based SCF, direct minimisation and Newton.

First we setup our problem

In [1]:
using DFTK
using LinearAlgebra

a = 10.26  # Silicon lattice constant in Bohr
lattice = a / 2 * [[0 1 1.];
                   [1 0 1.];
                   [1 1 0.]]
Si = ElementPsp(:Si, psp=load_psp("hgh/lda/Si-q4"))
atoms     = [Si, Si]
positions = [ones(3)/8, -ones(3)/8]

model = model_LDA(lattice, atoms, positions)
basis = PlaneWaveBasis(model; Ecut=5, kgrid=[3, 3, 3])

# Convergence we desire in the density
tol = 1e-6

1.0e-6

## Density-based self-consistent field

In [2]:
scfres_scf = self_consistent_field(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846705468839                   -0.70    4.5         
  2   -7.852299507379       -2.25       -1.54    1.0   22.5ms
  3   -7.852609027163       -3.51       -2.56    1.5   24.8ms
  4   -7.852645981234       -4.43       -2.87    2.5   30.6ms
  5   -7.852646476317       -6.31       -3.13    1.0   22.6ms
  6   -7.852646679705       -6.69       -4.18    1.0   23.1ms
  7   -7.852646686637       -8.16       -4.79    2.2   30.1ms
  8   -7.852646686722      -10.07       -5.48    1.2   24.0ms
  9   -7.852646686726      -11.39       -5.47    1.8   27.5ms
 10   -7.852646686730      -11.45       -6.79    1.0   23.1ms


## Potential-based SCF

In [3]:
scfres_scfv = DFTK.scf_potential_mixing(basis; tol);

n     Energy            log10(ΔE)   log10(Δρ)   α      Diag   Δtime
---   ---------------   ---------   ---------   ----   ----   ------
  1   -7.846861981055                   -0.70           4.2         
  2   -7.852526544895       -2.25       -1.64   0.80    2.2    251ms
  3   -7.852635484328       -3.96       -2.74   0.80    1.0   21.5ms
  4   -7.852646585303       -4.95       -3.23   0.80    2.0   28.0ms
  5   -7.852646681395       -7.02       -4.09   0.80    1.2   22.4ms
  6   -7.852646686340       -8.31       -4.93   0.80    1.8   24.4ms
  7   -7.852646686724       -9.42       -5.74   0.80    2.0   27.2ms
  8   -7.852646686730      -11.28       -7.02   0.80    2.2   27.8ms


## Direct minimization
Note: Unlike the other algorithms, tolerance for this one is in the energy,
thus we square the density tolerance value to be roughly equivalent.

In [4]:
scfres_dm = direct_minimization(basis; tol=tol^2);

Iter     Function value   Gradient norm 
     0     1.414416e+01     3.819027e+00
 * time: 0.06415700912475586
     1     1.499629e+00     1.816671e+00
 * time: 0.28557395935058594
     2    -1.828028e+00     2.140743e+00
 * time: 0.3086268901824951
     3    -3.781181e+00     1.699356e+00
 * time: 0.3420140743255615
     4    -5.284528e+00     1.673057e+00
 * time: 0.3753979206085205
     5    -6.781747e+00     1.205959e+00
 * time: 0.408519983291626
     6    -6.801508e+00     1.846747e+00
 * time: 0.49550294876098633
     7    -7.399865e+00     1.634452e+00
 * time: 0.5188210010528564
     8    -7.475928e+00     2.672984e+00
 * time: 0.5417439937591553
     9    -7.556175e+00     1.440332e+00
 * time: 0.5744199752807617
    10    -7.609637e+00     1.086797e+00
 * time: 0.6174969673156738
    11    -7.705436e+00     1.523459e+00
 * time: 0.6401748657226562
    12    -7.739166e+00     1.174956e+00
 * time: 0.6632978916168213
    13    -7.752216e+00     8.066082e-01
 * time: 0.69653105

## Newton algorithm

Start not too far from the solution to ensure convergence:
We run first a very crude SCF to get close and then switch to Newton.

In [5]:
scfres_start = self_consistent_field(basis; tol=0.5);

n     Energy            log10(ΔE)   log10(Δρ)   Diag   Δtime
---   ---------------   ---------   ---------   ----   ------
  1   -7.846841719537                   -0.70    4.8         


Remove the virtual orbitals (which Newton cannot treat yet)

In [6]:
ψ = DFTK.select_occupied_orbitals(basis, scfres_start.ψ, scfres_start.occupation).ψ
scfres_newton = newton(basis, ψ; tol);

n     Energy            log10(ΔE)   log10(Δρ)   Δtime
---   ---------------   ---------   ---------   ------
  1   -7.852645906125                   -1.64         
  2   -7.852646686730       -6.11       -3.71    1.85s
  3   -7.852646686730      -13.29       -7.25    145ms


## Comparison of results

In [7]:
println("|ρ_newton - ρ_scf|  = ", norm(scfres_newton.ρ - scfres_scf.ρ))
println("|ρ_newton - ρ_scfv| = ", norm(scfres_newton.ρ - scfres_scfv.ρ))
println("|ρ_newton - ρ_dm|   = ", norm(scfres_newton.ρ - scfres_dm.ρ))

|ρ_newton - ρ_scf|  = 4.285720935158202e-7
|ρ_newton - ρ_scfv| = 4.803478664928212e-7
|ρ_newton - ρ_dm|   = 1.5003720431345346e-9
